# Importing Modules

In [1]:
from tkinter import Tk, Label, Button, Canvas, Frame, StringVar
from tkinter import TOP, BOTTOM, LEFT, RIGHT, BOTH
import time
from PIL import ImageGrab

from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np

%matplotlib inline

# Loading Trained CNN Model

In [2]:
model = load_model("digits_trained_CNN.h5")

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

# UI using Tkinter

In [12]:
class DigitsGUI:
    def __init__(self, master):
        self.master = master
        
        self.header_frame = Frame(master)
        self.header_frame.pack(side = TOP)
        
        self.canvas_frame = Frame(master)
        self.canvas_frame.pack(side = TOP)
        
        self.bottom_frame = Frame(master)
        self.bottom_frame.pack(side = TOP)
        
        # Toggles when user clicks "Enter"
        self.should_draw = False
        self.default_result = """
        Result is:
        0  : {:2.2f}%
        1  : {:2.2f}%
        2  : {:2.2f}%
        3  : {:2.2f}%
        4  : {:2.2f}%
        5  : {:2.2f}%
        6  : {:2.2f}%
        7  : {:2.2f}%
        8  : {:2.2f}%
        9  : {:2.2f}%
        """
        self.answer = StringVar()
        self.answer.set(self.default_result.format(0,0,0,0,0,0,0,0,0,0))
        
        # Coordinate lists to draw
        self.coords = []
        
        master.title("Digits classifier UI test")
        
        # Event bindings
        master.bind('<Motion>', self.motion)
        master.bind('<Key>', self.check_enter_key)
  
        # Header label
        self.label = Label(self.header_frame, text="Digits UI\nPress Enter to start\nThen draw\nPress Enter to stop")
        self.label.pack(side = TOP)
        
        # Defining Canvas
        self.C = Canvas(self.canvas_frame, bg="white", height=280, width=280)
        self.C.pack(side = LEFT)
        
        # Result UI Display
        self.summary = Label(self.canvas_frame, textvariable = self.answer)
        self.summary.pack(side = RIGHT)

        # Clear the canvas button
        self.clear_button = Button(self.bottom_frame, text="Clear", command=self.clear)
        self.clear_button.pack(side = LEFT, expand = True, fill = BOTH)
        
        # Digit Classifier Result button
        self.result_button = Button(self.bottom_frame, text="Result", command=self.result)
        self.result_button.pack(side = LEFT, expand = True, fill = BOTH)
        
        # Close button
        self.close_button = Button(self.bottom_frame, text="Close", command=self.close)
        self.close_button.pack(side = LEFT, expand = True, fill = BOTH)

    # Toggles when user press enter key in keyboard
    def check_enter_key(self, event):
        if event.keycode == 13: # 13 is key code for "Enter"
            self.should_draw ^= True
            print("Toggled to: {}".format(self.should_draw))
            self.coords = []
    
    # Cleaning the canvas function
    def clear(self):
        print("Clearing!")
        self.C.delete("all")
        self.coords = []
        self.answer.set(self.default_result.format(0,0,0,0,0,0,0,0,0,0))
        
    # Classifier result function
    def result(self):
        print("Saving Image")
        self.getter()
        
        print("Loading image")
        img = cv2.imread("canvas_image.jpg")
        
        print("Preprocessing")
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        res = cv2.resize(gray, (28,28))
        res = res.astype(np.float64)
        res = res/255.0
        res = res.reshape((28,28,1))
        print(res.shape, res.dtype, res.max(), res.min())
        
        print("Prediction")
        prediction = model.predict(np.array([res]))
        print(prediction)
        self.answer.set(self.default_result.format(*prediction[0]))
        
    # Closing the GUI function
    def close(self):
        print("Closing!")
        self.master.destroy()
    
    # Saving the canvas as image on "Result" button click
    def getter(self):
        x=self.master.winfo_rootx() + self.C.winfo_x()
        y=self.master.winfo_rooty() + self.C.winfo_y()
        x1=x+self.C.winfo_width()
        y1=y+self.C.winfo_height()
        ImageGrab.grab().crop((x,y,x1,y1)).save("canvas_image.jpg")
        
    # Draws lines on canvas
    def motion(self, event):
        if self.should_draw == False:
            return
        time.sleep(0.25)
        x, y = event.x, event.y
        if self.coords == []:
            self.coords.extend([x,y, x,y])
        else:
            self.coords.extend([x,y])
        line = self.C.create_line(self.coords,fill='black', width=25)
        y = self.coords.pop()
        x = self.coords.pop()
        self.coords = [x,y]
        return
        

root = Tk()
my_gui = DigitsGUI(root)
root.mainloop()

Toggled to: True
Toggled to: False
Saving Image
Loading image
Preprocessing
(28, 28, 1) float64 1.0 0.0
Prediction
[[3.3591706e-03 2.0313693e-02 1.4013581e-05 2.1740301e-04 4.0625720e-04
  2.4919091e-03 2.5001691e-05 9.7052819e-01 2.9188683e-05 2.6151745e-03]]
Clearing!
Toggled to: True
Toggled to: False
Saving Image
Loading image
Preprocessing
(28, 28, 1) float64 1.0 0.0
Prediction
[[1.2994979e-03 6.0781866e-04 3.9871964e-05 2.3284843e-02 3.3051805e-03
  8.1956905e-01 1.1731150e-02 8.0867946e-02 2.1828562e-03 5.7111807e-02]]
Closing!
